In [3]:
import duckdb
import pandas as pd
import os

In [12]:
%env DUCKDB_DATABASE=staging/data.duckdb

env: DUCKDB_DATABASE=staging/data.duckdb


In [13]:
conn = duckdb.connect(os.getenv("DUCKDB_DATABASE"))

In [35]:
query = """
        select
            *
        from trips
        order by dropoff_datetime desc 
    """
df = conn.execute(query).fetch_df()
df.head()

,vendor_id,pickup_zone_id,dropoff_zone_id,rate_code_id,payment_type,dropoff_datetime,pickup_datetime,trip_distance,passenger_count,total_amount
0,2,233,158,1.0,1,2023-04-05 20:35:28,2023-04-05 20:17:42,3.33,1.0,26.10
1,2,230,162,1.0,2,2023-04-05 20:14:14,2023-04-05 20:05:04,0.95,1.0,14.00
2,2,144,140,1.0,1,2023-04-05 19:23:11,2023-04-05 19:07:21,3.57,2.0,30.12
3,2,142,144,1.0,1,2023-04-05 19:06:06,2023-04-05 18:36:18,4.41,2.0,39.84
4,2,236,239,1.0,1,2023-04-05 18:26:28,2023-04-05 18:18:38,1.60,2.0,16.50


In [34]:
query = """
        SELECT
            DATE_TRUNC('week', CAST(dropoff_datetime AS date) + 1) - 1 AS period,
            COUNT(*) AS num_trips,
            SUM(COALESCE(passenger_count,0)) AS passenger_count,
            SUM(COALESCE(total_amount,0)) AS total_amount,
            SUM(COALESCE(trip_distance,0)) AS trip_distance            
        FROM trips
        GROUP BY 1
        ORDER BY 1 
    """
trips_by_zone = conn.execute(query).fetch_df()
trips_by_zone.head(100)

,period,num_trips,passenger_count,total_amount,trip_distance
0,2000-12-31,3,5.0,1.717000e+02,34.22
1,2002-12-29,5,7.0,3.267000e+02,62.67
2,2008-12-28,2,2.0,6.595000e+01,14.33
3,2014-11-16,1,2.0,5.238000e+01,6.43
4,2023-02-19,55,54.0,9.109000e+02,0.01
5,2023-02-26,462235,604453.0,1.239400e+07,2028345.88
6,2023-03-05,776671,1019561.0,2.122563e+07,2735082.42
7,2023-03-12,781387,1037892.0,2.183735e+07,3042660.76
8,2023-03-19,741068,977864.0,2.086793e+07,2747952.94
9,2023-03-26,642270,841559.0,1.830822e+07,2733522.68


In [36]:
conn.close()